# Gesture Annotation Notebook |

##### <strong>Author:</strong> <u>Walter Dych</u> <em>(walterpdych@gmail.com)</em>
##### <strong>Edits/Documentation:</strong> <u>Karee Garvin</u> <em>(kgarvin@fas.harvard.edu)</em>

In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

In [2]:
data = pd.read_csv('C:/Users/cosmo/Desktop/Random Scripts/Co-Speech Gesture Automation/Co-Speech-Gesture-Automation/SPEED_FILES/5012_I_right_wrist_processed_data.csv')
data.head()

,right_shoulder_x,right_shoulder_y,left_shoulder_x,left_shoulder_y,right_elbow_x,right_elbow_y,left_elbow_x,left_elbow_y,right_wrist_x,right_wrist_y,...,left_wrist_y,right_eye_x,right_eye_y,left_eye_x,left_eye_y,nose_x,nose_y,speed_unsmooth,speed_smooth,time_ms
0,0.346543,0.413691,0.403167,0.370346,0.267567,0.594082,0.388862,0.499703,0.243763,0.848833,...,0.624571,0.442876,0.292149,0.445312,0.289609,0.447097,0.314755,NaN,NaN,0.0
1,0.347616,0.410525,0.403116,0.369793,0.267520,0.593730,0.388830,0.504563,0.243648,0.848302,...,0.635405,0.443190,0.292090,0.445706,0.289606,0.447535,0.314785,NaN,NaN,10.0
2,0.348689,0.407358,0.403065,0.369239,0.267474,0.593378,0.388798,0.509424,0.243532,0.847770,...,0.646239,0.443504,0.292031,0.446099,0.289603,0.447973,0.314815,NaN,NaN,20.0
3,0.349762,0.404192,0.403014,0.368685,0.267428,0.593026,0.388766,0.514284,0.243417,0.847239,...,0.657073,0.443817,0.291971,0.446493,0.289600,0.448412,0.314845,0.048880,NaN,30.0
4,0.350241,0.403980,0.402719,0.368353,0.267418,0.593071,0.386474,0.526024,0.243248,0.846921,...,0.674273,0.443986,0.292087,0.446667,0.289823,0.448469,0.315595,0.043379,NaN,40.0


In [3]:
MAX_S_LENGTH = 100
MAX_STEADY_LENGTH = 100
THRESHOLD = 0.4
COLORS = {'Stroke': 'rgba(231,107,243,0.2)', 'Rest': 'rgba(107,174,214,0.2)', '': 'rgba(255,255,255,0.2)'}

In [4]:
def generate_annotations(data): # Generate annotations based on speed_smooth data.
    speed_smooth = list(map(float, data['speed_smooth']))
    phase = [''] * len(speed_smooth)
    last_stroke_type = None
    start_index = None
    steady_start = None
    steady_end = None
    apexes = [''] * len(speed_smooth)

    # Iterate over the speed_smooth array to generate the annotations
    for i in range(1, len(speed_smooth)):
        if speed_smooth[i] > THRESHOLD and speed_smooth[i-1] <= THRESHOLD:
            if last_stroke_type is None or last_stroke_type == 'Steady' or last_stroke_type == 'R':
                last_stroke_type = 'Approach'
                start_index = i
            elif last_stroke_type == 'Approach' and i - start_index >= MAX_S_LENGTH:
                last_stroke_type = None
        elif speed_smooth[i] <= THRESHOLD and speed_smooth[i-1] > THRESHOLD:
            if last_stroke_type == 'Approach':
                last_stroke_type = 'Steady'
                start_index = i
                steady_start = i
            elif last_stroke_type == 'R':
                last_stroke_type = None
                start_index = None
        elif last_stroke_type == 'Steady' and speed_smooth[i] > THRESHOLD and speed_smooth[i-1] <= THRESHOLD:
            if steady_end is not None:
                last_stroke_type = 'R'
                start_index = i

        if last_stroke_type == 'Steady' and steady_start is None:
            steady_start = i
        elif steady_start is not None and last_stroke_type != 'Steady':
            steady_end = i
            min_speed_index = np.argmin(speed_smooth[steady_start:i]) + steady_start
            apexes[min_speed_index] = 'AX'
            steady_start = None
            if speed_smooth[i] > THRESHOLD and speed_smooth[i-1] <= THRESHOLD:
                last_stroke_type = 'R'
                start_index = i
        elif last_stroke_type == 'Steady' and i - start_index >= MAX_STEADY_LENGTH:
            last_stroke_type = None
            start_index = None
            steady_start = None
        phase[i] = last_stroke_type if last_stroke_type is not None else ''

    data = pd.DataFrame({
        'time_ms': data['time_ms'],
        'speed_unsmooth': data['speed_unsmooth'],
        'speed_smooth': data['speed_smooth'],
        'phase': phase, 
        'apex': apexes
    })

    data['phase'] = data['phase'].apply(lambda x: 'Stroke' if x in ['Steady', 'Approach'] else ('Rest' if x == 'R' else ''))
    return data



In [5]:
data = generate_annotations(data)
data.head()

,time_ms,speed_unsmooth,speed_smooth,phase,apex
0,0.0,NaN,NaN,,
1,10.0,NaN,NaN,,
2,20.0,NaN,NaN,,
3,30.0,0.048880,NaN,,
4,40.0,0.043379,NaN,,


In [6]:
data.to_csv('C:/Users/cosmo/Desktop/Random Scripts/Co-Speech Gesture Automation/Co-Speech-Gesture-Automation/ANNOTATIONS/5012_I_right_wrist_annotations.csv', index=False)

In [7]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=data['time_ms'], 
    y=data['speed_smooth'], 
    mode='lines',
    name='Speed curve'
))

data['phase_change'] = data['phase'].shift() != data['phase']
data['Group'] = data['phase_change'].cumsum()
interval_df = data.groupby(['Group', 'phase']).agg(Start=('time_ms', 'min'), End=('time_ms', 'max')).reset_index()

shapes = []
for _, row in interval_df.iterrows():
    shapes.append(dict(
        type="rect",
        xref="x", yref="paper",
        x0=row['Start'], x1=row['End'],
        y0=0, y1=1,
        fillcolor=COLORS[row['phase']],
        opacity=0.85,
        layer="below",
        line_width=0,
    ))

for phase in COLORS.keys():
    fig.add_trace(go.Scatter(
        x=[None], y=[None],
        mode='markers',
        marker=dict(size=15, color=COLORS[phase]),
        showlegend=True,
        name=phase,
    ))

apex_df = data[data['apex'] == 'AX']
fig.add_trace(go.Scatter(
    x=apex_df['time_ms'], 
    y=apex_df['speed_smooth'], 
    mode='markers',
    name='Apexes'
))

fig.update_xaxes(title_text='Time (ms)')
fig.update_yaxes(title_text='Speed (px/ms)')
fig.update_layout(showlegend=True, shapes=shapes)
fig.update_layout(title_text='Smoothed Speed Curve with Annotations')
fig.update_layout(width=1250, height=750)
fig.show()
